## The following shows fire brigade data for the last three years. This is an excel export from the fire brigade website

## import Packages

In [41]:
import folium
import pandas as pd
import requests

pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'

In [42]:
df = pd.read_csv("../data_output/incident_data.csv")

df["datetime"] = pd.to_datetime(df["datetime"])
df['mod_datetime_df'] = df['datetime'].dt.round('H')

## Cleaning the data
df.PropertyType = df.PropertyType.str.strip().str.lower()
# df.to_csv("/Users/meg.patakota/Library/CloudStorage/OneDrive-SharedLibraries-MishconDeReya/Data - DS&A Reports/Purpose/incident_data.xlsx")

"""Starting with HAVERING and RM13 District"""
df = df[(df["IncGeo_BoroughName"] == "HAVERING") & (df["Postcode_district"] == "RM13") & (df["PropertyType"] == "landfill site")]
print(df.shape)
df.head()

(123, 42)


,Unnamed: 0,IncidentNumber,DateOfCall,CalYear,TimeOfCall,HourOfCall,IncidentGroup,StopCodeDescription,SpecialServiceType,PropertyCategory,PropertyType,AddressQualifier,Postcode_full,Postcode_district,UPRN,USRN,IncGeo_BoroughCode,IncGeo_BoroughName,ProperCase,IncGeo_WardCode,IncGeo_WardName,IncGeo_WardNameNew,Easting_m,Northing_m,Easting_rounded,Northing_rounded,Latitude,Longitude,FRS,IncidentStationGround,FirstPumpArriving_AttendanceTime,FirstPumpArriving_DeployedFromStation,SecondPumpArriving_AttendanceTime,SecondPumpArriving_DeployedFromStation,NumStationsWithPumpsAttending,NumPumpsAttending,PumpCount,PumpHoursRoundUp,Notional Cost (£),NumCalls,datetime,mod_datetime_df
38693,38693,065008-30052020,2020-05-30,2020,03:09:30,3,Fire,Secondary Fire,NaN,Outdoor,landfill site,On land associated with building,RM13 9DB,RM13,10091579899,21300740,E09000016,HAVERING,Havering,E05013979,Rainham & Wennington,Rainham & Wennington,551237.0,181212.0,551250,181250,51.509605,0.177968,London,Wennington,602.0,Wennington,NaN,NaN,1.0,1.0,1.0,2.0,692.0,1.0,2020-05-30 03:09:30,2020-05-30 03:00:00
38906,38906,065428-30052020,2020-05-30,2020,19:38:55,19,Fire,Secondary Fire,NaN,Outdoor,landfill site,Open land/water - nearest gazetteer location,RM13 9BJ,RM13,10025586005,21300740,E09000016,HAVERING,Havering,E05013979,Rainham & Wennington,Rainham & Wennington,552623.0,179045.0,552650,179050,51.489766,0.196977,London,Wennington,600.0,Wennington,NaN,NaN,1.0,1.0,1.0,1.0,346.0,1.0,2020-05-30 19:38:55,2020-05-30 20:00:00
56979,56979,096747-04082020,2020-08-04,2020,00:30:12,0,Fire,Primary Fire,NaN,Outdoor,landfill site,Open land/water - nearest gazetteer location,RM13 9DS,RM13,10094416953,21320397,E09000016,HAVERING,Havering,E05013979,Rainham & Wennington,Rainham & Wennington,554142.0,181900.0,554150,181950,51.515002,0.220087,London,Wennington,269.0,Wennington,632.0,Dagenham,2.0,2.0,19.0,74.0,25604.0,2.0,2020-08-04 00:30:12,2020-08-04 01:00:00
61671,61671,105611-17082020,2020-08-17,2020,06:27:19,6,Fire,Secondary Fire,NaN,Outdoor,landfill site,Open land/water - nearest gazetteer location,RM13 9ST,RM13,10070703694,21300820,E09000016,HAVERING,Havering,E05013979,Rainham & Wennington,Rainham & Wennington,554209.0,181713.0,554250,181750,51.513309,0.220967,London,Wennington,422.0,Wennington,610.0,Dagenham,3.0,3.0,3.0,3.0,1038.0,1.0,2020-08-17 06:27:19,2020-08-17 06:00:00
64815,64815,110691-28082020,2020-08-28,2020,06:32:33,6,Fire,Secondary Fire,NaN,Outdoor,landfill site,Open land/water - nearest gazetteer location,RM13 9DS,RM13,10094416953,21320397,E09000016,HAVERING,Havering,E05013979,Rainham & Wennington,Rainham & Wennington,554121.0,181865.0,554150,181850,51.514700,0.219766,London,Wennington,267.0,Wennington,NaN,NaN,1.0,1.0,2.0,4.0,1384.0,1.0,2020-08-28 06:32:33,2020-08-28 07:00:00


### filter the dataframe by the post codes given by Mishcon Purpose

In [43]:
postcodes = ["RM13 9XD", "RM13 9NZ", "RM13 9LJ","RM13 9SA", "RM13 9SS", "RM13 9TA","RM13 9FL"]

In [44]:
def get_coordinates(postcodes):
    # API endpoint for bulk postcode lookup
    api_url = "https://api.postcodes.io/postcodes"
    
    # Make a POST request to the API with your list of postcodes
    response = requests.post(api_url, json={"postcodes": postcodes})
    
    # Check if the request was successful (HTTP Status Code 200)
    if response.status_code == 200:
        results = response.json()['result']
        # Extract and return the coordinates from the API response
        coordinates = {}
        for result in results:
            if result['result']:
                pc = result['query']
                lat = result['result']['latitude']
                lon = result['result']['longitude']
                coordinates[pc] = (lat, lon)
        return coordinates
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return None

coordinates = get_coordinates(postcodes) ## chosen postcodes from above
chosen_sites = pd.DataFrame(coordinates).T.reset_index().rename(columns={"index": "Postcode_full", 0: "Latitude", 1: "Longitude"})

min_lat , min_lon = chosen_sites.Latitude.min(), chosen_sites.Longitude.min()
max_lat , max_lon = chosen_sites.Latitude.max(), chosen_sites.Longitude.max()

## calculate the center of the map
center_lat, center_lon = (min_lat + max_lat) / 2, (min_lon + max_lon) / 2

# Create a map object centered around London.
m = folium.Map(location=[center_lat, center_lon], zoom_start=15, width='70%', height=500)

# Loop to add markers.
for _, row in chosen_sites.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup='A Pin!',
        icon=folium.Icon(icon='cloud'),
    ).add_to(m)

# Display the map in Jupyter Notebook.
m

## bringing back breath London data

In [45]:
df_bl = pd.read_csv("../data_output/breathe_london_data_main.csv")
df_bl.head()

,species,date,ScaledValue,Latitude,Longitude
0,INO2,2022-10-27,28.831556,51.520786,0.205461
1,INO2,2022-10-28,28.842309,51.520786,0.205461
2,INO2,2022-10-29,25.722402,51.520786,0.205461
3,INO2,2022-10-30,15.754954,51.520786,0.205461
4,INO2,2022-10-31,24.489301,51.520786,0.205461


### threshold information
- PM2.5 Daily (24 Hour) Average threshold is 15
- NO2 Daily (24 Hour) Average threshold 25

In [46]:
## when the spices is IPM25 and the ScaledValue is 35 or above, create a new column called 'alert'
## when the spices is INO2 and the ScaledValue is 40 or above, create a new column called 'alert'
df_bl["alert"] = df_bl.apply(
    lambda x: "alert"
    if (x["species"] == "IPM25" and x["ScaledValue"] >= 15)
    or (x["species"] == "INO2" and x["ScaledValue"] >= 25)
    else "no alert",
    axis=1,
)
df_bl.head()

,species,date,ScaledValue,Latitude,Longitude,alert
0,INO2,2022-10-27,28.831556,51.520786,0.205461,alert
1,INO2,2022-10-28,28.842309,51.520786,0.205461,alert
2,INO2,2022-10-29,25.722402,51.520786,0.205461,alert
3,INO2,2022-10-30,15.754954,51.520786,0.205461,no alert
4,INO2,2022-10-31,24.489301,51.520786,0.205461,no alert


In [47]:
'''Filter out the dataset to focus on only days where there was an alert'''
df_main_alert = df_bl[df_bl['alert'] == 'alert']
print(df_main_alert.shape)
'''Separting the alert dataset into two datasets, one for IPM25 and one for INO2'''
df_pm = df_main_alert[df_main_alert['species'] == 'IPM25']
df_no2 = df_main_alert[df_main_alert['species'] == 'INO2']
print(df_pm.shape), print(df_no2.shape)

(151, 6)
(45, 6)
(106, 6)


(None, None)

## Plotting alert areas with fire brigade data

In [48]:
def get_coordinates(postcodes):
    # API endpoint for bulk postcode lookup
    api_url = "https://api.postcodes.io/postcodes"
    
    # Make a POST request to the API with your list of postcodes
    response = requests.post(api_url, json={"postcodes": postcodes})
    
    # Check if the request was successful
    if response.status_code == 200:
        results = response.json()['result']
        # Extract and return the coordinates from the API response
        coordinates = {}
        for result in results:
            if result['result']:
                pc = result['query']
                lat = result['result']['latitude']
                lon = result['result']['longitude']
                coordinates[pc] = (lat, lon)
        return coordinates
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return None

# Replace 'postcodes_list' with your actual list of postcodes.
postcodes_list = postcodes
coordinates = get_coordinates(postcodes_list)

chosen_sites = pd.DataFrame(coordinates).T.reset_index().rename(columns={"index": "Postcode_full", 0: "Latitude", 1: "Longitude"})

min_lat, min_lon = chosen_sites.Latitude.min(), chosen_sites.Longitude.min()
max_lat, max_lon = chosen_sites.Latitude.max(), chosen_sites.Longitude.max()

# Calculate the center of the map
center_lat, center_lon = (min_lat + max_lat) / 2, (min_lon + max_lon) / 2

# Create a map object centered around the calculated center.
m = folium.Map(location=[center_lat, center_lon], zoom_start=15, width='70%', height=500)

# Add markers from the postcode API to the map.
for _, row in chosen_sites.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f'{row["Postcode_full"]}',
        icon=folium.Icon(icon='cloud'),
    ).add_to(m)

# Add markers for species locations.
for _, row in df_main_alert.iterrows():
    color = 'red' if row['species'] == 'INO2' else 'green'  # Assuming only two species INO2 and PM2.5

    popup_content = (f"Species: {row['species']}<br>"
                     f"Date: {row['date']}<br>"
                     f"ScaledValue: {row['ScaledValue']}<br>"
                     f"Alert: {row['alert']}")

    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=folium.Popup(popup_content, parse_html=True),
        icon=folium.Icon(color=color),
    ).add_to(m)

# Display the map in Jupyter Notebook or save it as an HTML file.
m

# To save the map as an HTML file, uncomment the following line:
# m.save('map.html')
